In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers


df = pd.read_csv("E:\Final year project\project\datapreprocessing\iot_combined.csv")


top_labels = {
    'PartOfAHorizontalPortScan': 1,
    'Benign': 0,
    'Okiru': 2,
    'DDoS': 3,
}

df = df[df['label'].isin(top_labels)]

balanced_df = pd.concat([df[df['label'] == label].sample(12500, replace=True) for label in top_labels])


balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)
balanced_df['label'] = balanced_df['label'].map(top_labels)

In [33]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


X = balanced_df.drop(columns=['label'])
y = balanced_df['label']


num_classes = 4
y_encoded = to_categorical(y, num_classes)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


input_shape_lstm = (X_train.shape[1], 1)
X_train_reshaped_lstm = X_train.values.reshape(X_train.shape[0], *input_shape_lstm)
X_test_reshaped_lstm = X_test.values.reshape(X_test.shape[0], *input_shape_lstm)


lstm_model = keras.Sequential([
    layers.LSTM(64, input_shape=input_shape_lstm),
    layers.Dense(num_classes, activation='softmax') 
])

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_reshaped_lstm, y_train, epochs=10, validation_data=(X_test_reshaped_lstm, y_test))


X_train_features_lstm = lstm_model.predict(X_train_reshaped_lstm)
X_test_features_lstm = lstm_model.predict(X_test_reshaped_lstm)



input_shape_cnn = (X_train_features_lstm.shape[1], 1)

X_train_features_reshaped_cnn = X_train_features_lstm.reshape(X_train_features_lstm.shape[0], *input_shape_cnn)
X_test_features_reshaped_cnn = X_test_features_lstm.reshape(X_test_features_lstm.shape[0], *input_shape_cnn)

num_classes = y_train.shape[1] 

cnn_model = keras.Sequential([
    layers.Conv1D(64, 3, activation='relu', input_shape=input_shape_cnn),
    layers.Conv1D(128, 3, activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_features_reshaped_cnn, y_train, epochs=10, validation_data=(X_test_features_reshaped_cnn, y_test))


Epoch 1/10
1250/1250 [==============================] - 44s 29ms/step - loss: 0.8063 - accuracy: 0.5997 - val_loss: 0.7231 - val_accuracy: 0.6825
Epoch 2/10
1250/1250 [==============================] - 33s 26ms/step - loss: 0.7598 - accuracy: 0.6239 - val_loss: 0.7201 - val_accuracy: 0.6487
Epoch 3/10
1250/1250 [==============================] - 41s 33ms/step - loss: 0.7554 - accuracy: 0.6186 - val_loss: 0.7165 - val_accuracy: 0.6486
Epoch 4/10
1250/1250 [==============================] - 38s 31ms/step - loss: 0.7633 - accuracy: 0.5993 - val_loss: 0.7225 - val_accuracy: 0.6474
Epoch 5/10
1250/1250 [==============================] - 37s 30ms/step - loss: 0.7374 - accuracy: 0.6332 - val_loss: 0.7640 - val_accuracy: 0.5976
Epoch 6/10
1250/1250 [==============================] - 38s 30ms/step - loss: 0.7374 - accuracy: 0.6272 - val_loss: 0.7189 - val_accuracy: 0.6488
Epoch 7/10
1250/1250 [==============================] - 39s 31ms/step - loss: 0.7289 - accuracy: 0.6397 - val_loss: 0.7249 -

ValueError: Exception encountered when calling layer "conv1d_33" (type Conv1D).

Negative dimension size caused by subtracting 3 from 2 for '{{node conv1d_33/Conv1D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](conv1d_33/Conv1D/ExpandDims, conv1d_33/Conv1D/ExpandDims_1)' with input shapes: [?,1,2,64], [1,3,64,128].

Call arguments received by layer "conv1d_33" (type Conv1D):
  • inputs=tf.Tensor(shape=(None, 2, 64), dtype=float32)

In [34]:
print(y_train.shape[1])

4


In [35]:
balanced_df

,Unnamed: 0,duration,orig_bytes,resp_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,537371,0.000000,0,0,0.0,0.0,0.0,0.0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,837280,0.000000,0,0,0.0,0.0,0.0,0.0,3,0,...,0,0,0,0,0,0,0,0,0,0
2,294668,3.159932,0,0,3.0,180.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1238401,0.000000,0,0,1.0,40.0,0.0,0.0,1,0,...,0,0,0,1,0,0,0,0,0,0
4,281011,0.000000,0,0,1.0,40.0,0.0,0.0,2,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,113342,0.000000,0,0,1.0,40.0,0.0,0.0,1,0,...,0,0,0,1,0,0,0,0,0,0
49996,1740861,0.000000,0,0,1.0,40.0,0.0,0.0,1,0,...,0,0,0,1,0,0,0,0,0,0
49997,921141,-1.000000,-1,-1,2.0,80.0,0.0,0.0,1,0,...,0,0,0,1,0,0,0,0,0,0
49998,842217,-1.000000,-1,-1,2.0,80.0,0.0,0.0,2,0,...,0,0,0,1,0,0,0,0,0,0
